# Adjust info

This notebook is used to consider late entries, reschedule matches, and move players from and into subteams.

In [1]:
from helper_functions import setup
import numpy as np
import matplotlib.pyplot as plt
import helper_functions as hf
from helper_functions.setup.team_creation import find_best_team_to_join
from helper_functions.setup.openai_image_download import generate_all_images, save_resized_animal_images


## Register latecomers

Add all latecoming players, and add them as a reserve in each subteam for the best-fitting team.

In [2]:
player_df = setup.sanitize_and_anonymize_data(overwrite=True, verbose=False)
late = player_df[player_df["late_entry"]]
teams = hf.get_teams()
subteams = hf.ALL_SUBTEAMS.values()

for _, player in late.iterrows():
    name = player["nickname"]
    # Don't do anything if the player has already been added
    if any([team.contains_player(name) for team in teams]):
        continue
    team_to_join = teams[find_best_team_to_join(teams, player)]
    team_to_join.add_player(player, register_as_reserve=True)
    print(f"{name} joins {team_to_join.name}")
    generate_all_images([name])
    save_resized_animal_images(150)


## Move players from reserve to main team

If players are attending a sport, even if just in reserve, and are substituted in, they should be registered there.

In [6]:
sport = "running_sprints"
player_name = "Pushy Bulldog"

def move_player_from_reserve_to_subteam(player_name: str, new_subteam: int | str, sport: str, save=True):
    """Move a player from the reserve subteam in the given sport to the subteam with the provided key."""
    player_df = hf.get_players().set_index("nickname", drop=False)
    current_subteam = player_df.loc[player_name, f"subteam_{sport}"]
    player_team: str = player_df.loc[player_name, "Team"]  # type: ignore
    team_letter = player_team.replace("Team ", "")
    assert f"{sport}_{team_letter}: {new_subteam}" in hf.ALL_SUBTEAMS
    if current_subteam != "R":
        print(f"The player you've chosen ({player_name}) is currently not in a reserve team for {sport}, skipping.")
        return
    print(f"Setting new subteam '{new_subteam}' in {sport} for {player_name}")
    player_df.loc[player_name, f"subteam_{sport}"] = new_subteam
    if save:
        fpath = hf.Team.backup_path_from_letter(team_letter)
        player_df[player_df["Team"] == player_team].to_csv(fpath, index=False)
move_player_from_reserve_to_subteam("Magnificent Barracuda", 2, "volleyball")


The player you've chosen (Magnificent Barracuda) is currently not in a reserve team for volleyball, skipping.
